In [ ]:
import os
import sys
import yaml
import time

import numpy as np
import geopandas as gpd
import pandas as pd
import pickle

from network_wrangler import (
    RoadwayNetwork,
    TransitNetwork,
    ProjectCard,
    Scenario,
    WranglerLogger,
)
from lasso import ModelRoadwayNetwork, StandardTransit, Parameters
from lasso import Project
from lasso.seflorida import seflorida

import logging

In [ ]:
print("setup basic paths...")
root_dir = "serpm/"
input_dir = os.path.join(root_dir, "data", "interim", "8-standard-format")
output_dir = os.path.join(root_dir, "data", "processed")
lasso_dir = "Lasso/"
card_dir = os.path.join(root_dir, "data", "project_cards")

print("read parameters...")
parameters = Parameters(lasso_base_dir=lasso_dir)

In [ ]:
print("read pickle...")
version_01_pickle_file_name = f"{output_dir}/working_scenario_01.pickle"
v_01_scenario = pickle.load(open(version_01_pickle_file_name, "rb"))

## Create Scenario and Apply Project Cards

In [ ]:
# create scenario
print("create scenario...")
v_01_scenario.road_net.links_df["id"] = v_01_scenario.road_net.links_df[
    "shstGeometryId"
]

step1_scenario = Scenario.create_scenario(
    base_scenario=v_01_scenario,
    card_directory=card_dir,
    tags=["year 2019"],
    validate_project_cards=False,
)

In [ ]:
step1_scenario.apply_all_projects()

In [ ]:
step1_scenario.applied_projects

## Make Travel Model Networks

In [ ]:
# Make Travel Model Networks
print("create travel model network...")
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object=step1_scenario.road_net, parameters=parameters
)

print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

In [ ]:
## add centroid & connector
model_net = seflorida.add_centroid_and_centroid_connector(
    roadway_network = model_net,
    parameters = parameters)

In [ ]:
print("convert standard roadway to seflorida network...")
model_net = seflorida.roadway_standard_to_seflorida_network(model_net, parameters, output_proj=2236)

In [ ]:
model_net.links_seflorida_df = model_net.links_df.copy()
model_net.nodes_seflorida_df = model_net.nodes_df.copy()

## Export results

In [ ]:
print("write as fixedwidth...")
model_net.write_roadway_as_fixedwidth(
    output_link_txt=os.path.join(output_dir, "links.txt"),
    output_node_txt=os.path.join(output_dir, "nodes.txt"),
    output_link_header_width_txt=os.path.join(output_dir, "links_header_width.txt"),
    output_node_header_width_txt=os.path.join(output_dir, "nodes_header_width.txt"),
    output_cube_network_script=os.path.join(
        output_dir, "make_complete_network_from_fixed_width_file.s"
    ),
)

In [ ]:
model_net.write_roadway_as_shp(
    output_link_shp=os.path.join(output_dir, "links.shp"),
    output_node_shp=os.path.join(output_dir, "nodes.shp"),
    link_output_variables=[
        "shstReferenceId",
        "model_link_id",
        "A",
        "B",
        "name",
        "oneWay",
        "roadway",
        "lanes_EA",
        "lanes_AM",
        "lanes_MD",
        "lanes_PM",
        "lanes_EV",
        "drive_access",
        "walk_access",
        "bike_access",
        "ftype",
        "managed",
        "CARTOLL",
        "geometry",
    ],
    node_output_variables=[
        "N",
        "model_node_id", 
        "county", 
        "drive_access", 
        "walk_access", 
        "bike_access", 
        "rail_only", 
        "farezone", 
        "X", 
        "Y", 
        "geometry"
    ],
    data_to_csv=False,
    data_to_dbf=True,
)

#### save pickle file

In [ ]:
v2_scenario = {"road_net": model_net, "transit_net": v_01_scenario.transit_net}
working_scenario = Scenario.create_scenario(base_scenario = v2_scenario)

working_scenario_filename = os.path.join(output_dir, 'working_scenario_02.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))